## Задача - сделать модель что будет угадывать по напитку угадывать средний балл

- достать данные
- разметить их
- обучить модель
- дать доступ через телеграмм бот

Исследование не претендует на научность - главная задача - развлечение

данные были собраны через Google Forms - я раскидал по чатам всех курсов ссылку на опрос результат сбора в табличке: - https://docs.google.com/spreadsheets/d/1GqODes6YSPXHsVeRFff1qCBORaGndstBX9WmngM2n1A/edit?gid=149531268#gid=149531268

чтобы удобно экспортировать в csv и в процессе добавления ответов добавлять новые результаты быстро написал скрипт экспорта в csv для google apps script

```
function exportFilteredCSV() {
  var startDate = new Date('2025-10-01'); // дата 

  var spreadsheet = SpreadsheetApp.getActiveSpreadsheet();
  var sheet = spreadsheet.getSheetByName('Ответы');
  var data = sheet.getDataRange().getValues();
  
  var dateColumnIndex = 0;
  // Оставляем заголовки и фильтруем строки по дате
  var headers = data[0];
  var filteredData = data.filter(function(row, index) {
    if (index === 0) return true;
    var rowDate = new Date(row[dateColumnIndex]);
    return rowDate > startDate;
  });
  
  // Преобразуем отфильтрованные данные в CSV-текст
  var csvContent = filteredData.map(function(row) {
    return row.map(function(cell) {
      cell = cell === null ? '' : cell.toString();
      if (cell.includes(',') || cell.includes('"') || cell.includes('\n')) {
        cell = '"' + cell.replace(/"/g, '""') + '"';
      }
      return cell;
    }).join(',');
  }).join('\n');
  
  var blob = Utilities.newBlob(csvContent, 'text/csv', 'новые_данные.csv');
  var file = DriveApp.createFile(blob);
  Logger.log('Файл создан: ' + file.getUrl());
}
```


Мы будем добавлять новые данные к старым данным - ручками оставляя релевантные данные

In [2]:
import pandas as pd

# === 1. Load the existing drinks.csv to get filter mappings ===
try:
    existing_drinks_df = pd.read_csv('drinks.csv')
    
    # Create case-insensitive mappings from existing data
    favorite_mapping = {}
    often_mapping = {}
    
    for _, row in existing_drinks_df.iterrows():
        if pd.notna(row['like_drink']) and pd.notna(row['like_drink_f']):
            key = str(row['like_drink']).lower().strip()
            favorite_mapping[key] = row['like_drink_f']
        
        if pd.notna(row['often_drink']) and pd.notna(row['often_drink_f']):
            key = str(row['often_drink']).lower().strip()
            often_mapping[key] = row['often_drink_f']
    
except FileNotFoundError:
    print("File drinks.csv not found. Starting from scratch.")
    existing_drinks_df = pd.DataFrame()
    favorite_mapping = {}
    often_mapping = {}

# === 2. Load and process the new_data.csv ===
new_data_df = pd.read_csv('new_data.csv')

# Function to convert timestamp to UNIX format
def convert_to_unix(timestamp_str):
    date_str = timestamp_str[:33]
    return pd.Timestamp(date_str).timestamp()

new_data_df['timestamp_unix'] = new_data_df['Отметка времени'].apply(convert_to_unix)

# Function to clean the 'money' column
def clean_money(value):
    if pd.isna(value) or value == '-':
        return ''
    value_str = str(value).lower().replace('k', '000').replace('к', '000')
    value_str = ''.join(char for char in value_str if char.isdigit())
    return value_str

new_data_df['money'] = new_data_df['money'].apply(clean_money)

# Drop the original timestamp column
new_data_df.drop(columns=['Отметка времени'], inplace=True)

# === 3. Create and populate the new filter columns using the mappings ===
new_data_df['like_drink_f'] = None
new_data_df['often_drink_f'] = None

# Temporary storage for new mappings from current session
new_favorite_mapping = favorite_mapping.copy()
new_often_mapping = often_mapping.copy()

for index, row in new_data_df.iterrows():
    # For favorite drink
    if pd.notna(row['like_drink']):
        like_drink_key = str(row['like_drink']).lower().strip()
        
        if like_drink_key in new_favorite_mapping:
            # Use existing mapping
            new_data_df.at[index, 'like_drink_f'] = new_favorite_mapping[like_drink_key]
        else:
            # Ask for input and add to mapping
            new_drink = input(f"Введите любимый напиток для строки {index} (текущее: {row['like_drink']}): ")
            new_data_df.at[index, 'like_drink_f'] = new_drink
            new_favorite_mapping[like_drink_key] = new_drink
    
    # For frequent drink
    if pd.notna(row['often_drink']):
        often_drink_key = str(row['often_drink']).lower().strip()
        
        if often_drink_key in new_often_mapping:
            # Use existing mapping
            new_data_df.at[index, 'often_drink_f'] = new_often_mapping[often_drink_key]
        else:
            # Ask for input and add to mapping
            new_drink = input(f"Введите частый напиток для строки {index} (текущее: {row['often_drink']}): ")
            new_data_df.at[index, 'often_drink_f'] = new_drink
            new_often_mapping[often_drink_key] = new_drink

# === 4. Combine with existing data and remove duplicates ===
# Check if we have existing data to combine with
if not existing_drinks_df.empty:
    # Remove from new_data_df any rows that have timestamps already in existing_drinks_df
    existing_timestamps = set(existing_drinks_df['timestamp_unix'])
    new_data_df = new_data_df[~new_data_df['timestamp_unix'].isin(existing_timestamps)]
    
    # Concatenate only if there are new rows
    if not new_data_df.empty:
        combined_df = pd.concat([existing_drinks_df, new_data_df], ignore_index=True)
    else:
        combined_df = existing_drinks_df
        print("No new data to add - all timestamps already exist.")
else:
    combined_df = new_data_df

# Reset the index for a clean order
combined_df.reset_index(drop=True, inplace=True)

# === 5. Save the updated combined dataset ===
combined_df.to_csv('drinks.csv', index=False)

print(f"Data has been successfully updated and saved to drinks.csv")
print(f"Total records: {len(combined_df)}")

Data has been successfully updated and saved to drinks.csv
Total records: 88


обучим модель взял XGBOOST по идее catboost бы лучше отработал но данные мусорные см R2 - фактически мы по приколу обучили генератор рандомных чисел жаль( я ожидал от любителей пива 5.0

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import joblib
import re

# Загрузка данных
df = pd.read_csv('drinks.csv')

# Функция для очистки балла
def clean_numeric_value(value, default=4.0):
    if pd.isna(value) or value == '':
        return default
    value_str = str(value).strip().replace(',', '.')
    cleaned = re.sub(r'[^\d\.]', '', value_str)
    
    try:
        result = float(cleaned)
        # Ограничиваем значение в диапазоне от 2 до 5 от умников
        return max(2.0, min(5.0, result))
    except ValueError:
        return default

# Функция для очистки категориальных значений
def clean_categorical_value(value):
    if pd.isna(value):
        return None
    cleaned = str(value).strip().lower()
    return cleaned if cleaned != '' else None

# Очищаем категориальные столбцы
categorical_columns = ['tier', 'like_drink_f', 'often_drink_f']
for col in categorical_columns:
    df[col] = df[col].apply(clean_categorical_value)

# Очищаем числовые столбцы
numeric_columns = ['mark']
for col in numeric_columns:
    df[col] = df[col].apply(clean_numeric_value)

print(f"Размер данных до удаления пропусков: {len(df)}")
# Удаляем строки, где пропущены категориальные признаки или целевая переменная
df = df.dropna(subset=categorical_columns + numeric_columns)

print(f"Размер данных после удаления пропусков: {len(df)}")

# Векторизация категориальных признаков с помощью LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col + '_encoded'] = le.fit_transform(df[col])
    label_encoders[col] = le

# Определение признаков и целевой переменной
features = ['tier_encoded', 'frequency_day', 'frequency_day_o', 'like_drink_f_encoded', 'often_drink_f_encoded']
X = df[features]
y = df['mark']

# Проверяем типы данных
print(f"Типы данных в X:")
print(X.dtypes)
print(f"Тип данных в y: {y.dtype}")

# Разделение на обучающую и тестовую выборки чтобы отследить как обучилась скорее по фолдам поймём чисто символически оставляю пару строк
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=52)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")
    

# K-FOLD КРОСС-ВАЛИДАЦИЯ ДЛЯ ПОИСКА ЛУЧШЕЙ МОДЕЛИ
def clip_predictions(predictions):
    return np.clip(predictions, 2.0, 5.0)

# Настройки K-Fold
n_splits = min(5, len(X_train) - 1)  # Адаптируем количество фолдов под объем данных
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Параметры для перебора
param_combinations = [
    {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 4},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 6},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 4},
    {'n_estimators': 5000, 'learning_rate': 0.1, 'max_depth': 20}, # по приколу
]

best_model = None
best_mse = np.inf
best_params = None
best_fold_results = []

print(f"\nНачинаем K-Fold кросс-валидацию ({n_splits} фолдов)...")

for param_idx, params in enumerate(param_combinations):
    print(f"\nТестируем комбинацию параметров {param_idx + 1}/{len(param_combinations)}: {params}")

    fold_scores = []
    fold_models = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = XGBRegressor(**params, random_state=52)
        model.fit(X_fold_train, y_fold_train)
        
        y_fold_pred = model.predict(X_fold_val)
        y_fold_pred_clipped = clip_predictions(y_fold_pred)
        
        fold_r2 = r2_score(y_fold_val, y_fold_pred_clipped)
        fold_mse = mean_squared_error(y_fold_val, y_fold_pred_clipped)
        
        fold_scores.append({'fold': fold, 'r2': fold_r2, 'mse': fold_mse})
        fold_models.append(model)
        
        print(f"Фолд {fold + 1}: MSE = {fold_mse:.4f}, R2 = {fold_r2:.4f}")
    
    avg_mse = np.mean([score['mse'] for score in fold_scores])
    avg_r2 = np.mean([score['r2'] for score in fold_scores])
    
    print(f"Средние по фолдам: MSE = {avg_mse:.4f}, R2 = {avg_r2:.4f}")
    
    if avg_mse < best_mse:
        best_mse = avg_mse
        best_params = params
        best_fold_idx = np.argmin([score['mse'] for score in fold_scores])
        best_model = fold_models[best_fold_idx]
        best_fold_results = fold_scores
        print(f"НОВАЯ ЛУЧШАЯ МОДЕЛЬ! MSE = {avg_mse:.4f}")

print("\nФИНАЛЬНАЯ ОЦЕНКА ЛУЧШЕЙ МОДЕЛИ")
print(f"Лучшие параметры: {best_params}")
print(f"Средний MSE на кросс-валидации: {best_mse:.4f}")

y_pred = best_model.predict(X_test)
y_pred_clipped = clip_predictions(y_pred)

final_mse = mean_squared_error(y_test, y_pred_clipped)
final_r2 = r2_score(y_test, y_pred_clipped)

print(f"\nРезультаты на тестовых данных:")
print(f"Среднеквадратичная ошибка (MSE): {final_mse:.4f}")
print(f"Коэффициент детерминации (R2): {final_r2:.4f}")
print(f"Диапазон предсказаний: от {y_pred_clipped.min():.2f} до {y_pred_clipped.max():.2f}")

# Сохраняем модель и энкодеры
best_model.save_model('best_xgboost_model.json')
joblib.dump(best_model, 'best_xgboost_model.joblib')

model_info = {
    'best_params': best_params,
    'test_r2': float(final_r2),
    'test_mse': float(final_mse),
    'features': features,
}

joblib.dump(model_info, 'model_info.joblib')
joblib.dump(label_encoders, 'label_encoders.joblib')

print("\nМодель успешно сохранена!")
print("best_xgboost_model.json - основная модель")
print("best_xgboost_model.joblib - модель в joblib формате")
print("model_info.joblib - информация о модели")
print("label_encoders.joblib - кодировщики категориальных признаков")

print(f"Процесс завершен! Лучшая модель сохранена с MSE = {final_mse:.4f} на тестовых данных")

Размер данных до удаления пропусков: 88
Размер данных после удаления пропусков: 86
Типы данных в X:
tier_encoded             int64
frequency_day            int64
frequency_day_o          int64
like_drink_f_encoded     int64
often_drink_f_encoded    int64
dtype: object
Тип данных в y: float64
Размер обучающей выборки: (81, 5)
Размер тестовой выборки: (5, 5)

Начинаем K-Fold кросс-валидацию (5 фолдов)...

Тестируем комбинацию параметров 1/4: {'n_estimators': 50, 'learning_rate': 0.1, 'max_depth': 4}
Фолд 1: MSE = 0.6884, R2 = -0.3411
Фолд 2: MSE = 0.4283, R2 = -0.7611
Фолд 3: MSE = 0.4031, R2 = 0.0907
Фолд 4: MSE = 0.7796, R2 = -0.4005
Фолд 5: MSE = 0.6879, R2 = -0.7268
Средние по фолдам: MSE = 0.5974, R2 = -0.4278
НОВАЯ ЛУЧШАЯ МОДЕЛЬ! MSE = 0.5974

Тестируем комбинацию параметров 2/4: {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 6}
Фолд 1: MSE = 0.6209, R2 = -0.2097
Фолд 2: MSE = 0.5373, R2 = -1.2090
Фолд 3: MSE = 0.4298, R2 = 0.0304
Фолд 4: MSE = 0.7731, R2 = -0.3888
Фолд 5

ну раз сделали модель надо её запихнуть в прод в тг бот смеха ради

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import re
import joblib

def clean_numeric_value(value, default=4.0):
    if pd.isna(value) or value == '':
        return default
    value_str = str(value).strip().replace(',', '.')
    cleaned = re.sub(r'[^\d\.]', '', value_str)
    try:
        result = float(cleaned)
        return max(2.0, min(5.0, result))
    except ValueError:
        return default

def clean_categorical_value(value):
    if pd.isna(value):
        return None
    return str(value).strip().lower()

# Загрузка модели из JSON
def load_xgboost_model(model_path='best_xgboost_model.json'):
    model = XGBRegressor()
    model.load_model(model_path)
    print(f"Модель загружена из {model_path}")
    return model

# Загрузка LabelEncoders
def load_label_encoders(encoders_path='label_encoders.joblib'):
    label_encoders = joblib.load(encoders_path)
    print(f"LabelEncoders загружены из {encoders_path}")
    return label_encoders

# Основная функция предсказания
def predict_mark(model, label_encoders, tier, frequency_day, frequency_day_o, like_drink_f, often_drink_f):
    # Очищаем входные данные
    tier_clean = clean_categorical_value(tier)
    frequency_day_clean = clean_numeric_value(frequency_day)
    frequency_day_o_clean = clean_numeric_value(frequency_day_o)
    like_drink_f_clean = clean_categorical_value(like_drink_f)
    often_drink_f_clean = clean_categorical_value(often_drink_f)
    
    # Создаем DataFrame для новых данных
    new_data = pd.DataFrame({
        'tier': [tier_clean],
        'frequency_day': [frequency_day_clean],
        'frequency_day_o': [frequency_day_o_clean],
        'like_drink_f': [like_drink_f_clean],
        'often_drink_f': [often_drink_f_clean]
    })
    
    categorical_columns = ['tier', 'like_drink_f', 'often_drink_f']
    
    for col in categorical_columns:
        le = label_encoders[col]
        try:
            new_data[col + '_encoded'] = le.transform(new_data[col])
        except ValueError:
            # Если значение новое, используем наиболее частый класс (0)
            new_data[col + '_encoded'] = 0
            print(f"Новое значение '{new_data[col].iloc[0]}' для признака '{col}'")
    
    # Используем закодированные признаки
    features = ['tier_encoded', 'frequency_day', 'frequency_day_o', 'like_drink_f_encoded', 'often_drink_f_encoded']
    features_new = new_data[features]
    
    # Предсказываем и ограничиваем результат
    prediction = model.predict(features_new)[0]
    clipped_prediction = max(2.0, min(5.0, prediction))
    
    return round(clipped_prediction, 2)

class MarkPredictor:
    def __init__(self, model_path='best_xgboost_model.json', encoders_path='label_encoders.joblib'):
        self.model = load_xgboost_model(model_path)
        self.label_encoders = load_label_encoders(encoders_path)
    
    def predict(self, tier, frequency_day, frequency_day_o, like_drink_f, often_drink_f):
        return predict_mark(self.model, self.label_encoders, tier, frequency_day, frequency_day_o, like_drink_f, often_drink_f)